# Importing libraries

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

# Importing data

In [35]:
# Importing LightGBM results
LGBM_no_sent = pd.read_csv('../datasets/processed_data/model_predictions/LightGBM/woSentiment/IPCALAB.BO.csv')
LGBM_w_sent = pd.read_csv('../datasets/processed_data/model_predictions/LightGBM/wSentiment/IPCALAB.BO.csv')

In [36]:
# Importing Prophet results
Prophet = pd.read_csv('../datasets/processed_data/model_predictions/Prophet/IPCALAB.BO.csv')

In [37]:
# Importing RF results
RF_no_sent = pd.read_csv('random_forest/rf_IPCALAB_without_sentiment.csv')
RF_w_sent = pd.read_csv('random_forest/rf_IPCALAB.csv')

In [38]:
# Importing LSTM results
LSTM = pd.read_csv('../datasets/processed_data/model_predictions/LSTM/IPCALAB_LSTM_predictions.csv')

In [39]:
# Linear Regression results
LR_no_sent = pd.read_csv('linear_regression/lr_IPCALAB_without_sentiment.csv')
LR_w_sent = pd.read_csv('linear_regression/lr_IPCALAB.csv')

# Combining datasets

## LSTM predictions outcome

In [40]:
LSTM.shape

(728, 4)

In [41]:
LSTM.head()

,date,Actual High,Predicted high LSTM (no sentiments),Predicted high LSTM (with sentiments)
0,2020-06-23,837.500000,842.837268,848.951481
1,2020-06-24,836.750000,852.151857,865.456525
2,2020-06-25,842.500000,857.745454,856.004721
3,2020-06-26,854.575012,860.110194,863.111140
4,2020-06-29,853.799988,874.979021,879.640504


## LGBM predictions outcome

In [42]:
LGBM_no_sent.head(2)

,date,high,pred_high
0,2020-06-23,837.50,834.781
1,2020-06-24,836.75,844.789


In [43]:
LGBM_no_sent.columns = ['date', 'high', 'Predicted high LGBM (no sentiments)']

In [44]:
LGBM_no_sent.shape

(359, 3)

In [45]:
LGBM_w_sent.head(2)

,date,high,pred_high
0,2020-06-23,837.50,835.679
1,2020-06-24,836.75,849.424


In [46]:
LGBM_w_sent.columns = ['date', 'high', 'Predicted high LGBM (with sentiments)']

In [47]:
LGBM_w_sent.shape

(359, 3)

## Prophet predictions outcome

In [48]:
Prophet.head(2)

,date,high,pred_high_lower,pred_high_upper,pred_high
0,2021-11-30,1078.099976,1055.193,1089.188,1071.796
1,2021-12-01,1072.500000,1040.496,1074.124,1056.460


In [49]:
Prophet.columns= ['date', 'high', 'pred_high_lower_Prophet', 'pred_high_upper_Prophet', 'Predicted high Prophet']

In [50]:
Prophet.shape

(369, 5)

## RF predictions outcome

In [51]:
RF_no_sent.head(2)

,date,y_test,y_pred
0,2020-06-23,837.50,848.022767
1,2020-06-24,836.75,853.722226


In [52]:
RF_no_sent.columns = ['date', 'y_test', 'Predicted high RF (no sentiments)']

In [53]:
RF_no_sent.shape

(359, 3)

In [54]:
RF_w_sent.columns = ['date', 'y_test', 'Predicted high RF (with sentiments)']

In [55]:
RF_w_sent.head(2)

,date,y_test,Predicted high RF (with sentiments)
0,2020-06-23,837.50,846.034409
1,2020-06-24,836.75,848.152623


In [56]:
RF_w_sent.shape

(728, 3)

## Linear regression predictions outcome

In [57]:
LR_no_sent.head(2)

,date,y_test,y_pred
0,2020-06-23,837.50,840.902375
1,2020-06-24,836.75,853.160294


In [58]:
LR_no_sent.columns = ['date', 'high', 'Predicted high LR (no sentiments)']

In [59]:
LR_w_sent.head(2)

,date,y_test,y_pred
0,2020-06-23,837.50,843.308882
1,2020-06-24,836.75,846.193335


In [60]:
LR_w_sent.columns = ['date', 'high', 'Predicted high LR (with sentiments)']

## Combining results into dataframe

In [61]:
df = pd.concat([RF_no_sent, RF_w_sent['Predicted high RF (with sentiments)'], 
                LGBM_no_sent['Predicted high LGBM (no sentiments)'], 
                LGBM_w_sent['Predicted high LGBM (with sentiments)'],
                LR_no_sent['Predicted high LR (no sentiments)'],
                LR_w_sent['Predicted high LR (with sentiments)']], axis=1)

In [62]:
df.head()

,date,y_test,Predicted high RF (no sentiments),Predicted high RF (with sentiments),Predicted high LGBM (no sentiments),Predicted high LGBM (with sentiments),Predicted high LR (no sentiments),Predicted high LR (with sentiments)
0,2020-06-23,837.500000,848.022767,846.034409,834.781,835.679,840.902375,843.308882
1,2020-06-24,836.750000,853.722226,848.152623,844.789,849.424,853.160294,846.193335
2,2020-06-25,842.500000,841.937978,842.960338,837.163,847.447,836.229378,842.844956
3,2020-06-26,854.575012,850.796238,850.550175,848.979,851.189,847.617173,848.446461
4,2020-06-29,853.799988,865.689731,864.928157,872.542,881.261,858.143265,857.283979


In [63]:
# Merging results with the LSTM results
result = pd.merge(df, LSTM, how="left", on=["date", "date"])

In [64]:
result = result.drop('y_test', axis=1)

In [65]:
result.head()

,date,Predicted high RF (no sentiments),Predicted high RF (with sentiments),Predicted high LGBM (no sentiments),Predicted high LGBM (with sentiments),Predicted high LR (no sentiments),Predicted high LR (with sentiments),Actual High,Predicted high LSTM (no sentiments),Predicted high LSTM (with sentiments)
0,2020-06-23,848.022767,846.034409,834.781,835.679,840.902375,843.308882,837.500000,842.837268,848.951481
1,2020-06-24,853.722226,848.152623,844.789,849.424,853.160294,846.193335,836.750000,852.151857,865.456525
2,2020-06-25,841.937978,842.960338,837.163,847.447,836.229378,842.844956,842.500000,857.745454,856.004721
3,2020-06-26,850.796238,850.550175,848.979,851.189,847.617173,848.446461,854.575012,860.110194,863.111140
4,2020-06-29,865.689731,864.928157,872.542,881.261,858.143265,857.283979,853.799988,874.979021,879.640504


In [66]:
result.columns

Index(['date', 'Predicted high RF (no sentiments)',
       'Predicted high RF (with sentiments)',
       'Predicted high LGBM (no sentiments)',
       'Predicted high LGBM (with sentiments)',
       'Predicted high LR (no sentiments)',
       'Predicted high LR (with sentiments)', 'Actual High',
       'Predicted high LSTM (no sentiments)',
       'Predicted high LSTM (with sentiments)'],
      dtype='object')

In [67]:
melted_df = pd.melt(result, id_vars =['date'],
                     value_vars =['Actual High','Predicted high LSTM (no sentiments)',
                                  'Predicted high LSTM (with sentiments)',
                                 'Predicted high RF (no sentiments)', 
                                  'Predicted high RF (with sentiments)', 
                                  'Predicted high LGBM (no sentiments)', 
                                  'Predicted high LGBM (with sentiments)',
                                 'Predicted high LR (no sentiments)',
                                 'Predicted high LR (with sentiments)'],
                    var_name ='Method', value_name ='High price')

In [68]:
melted_df.head(5)

,date,Method,High price
0,2020-06-23,Actual High,837.500000
1,2020-06-24,Actual High,836.750000
2,2020-06-25,Actual High,842.500000
3,2020-06-26,Actual High,854.575012
4,2020-06-29,Actual High,853.799988


In [69]:
melted_df.Method.unique()

array(['Actual High', 'Predicted high LSTM (no sentiments)',
       'Predicted high LSTM (with sentiments)',
       'Predicted high RF (no sentiments)',
       'Predicted high RF (with sentiments)',
       'Predicted high LGBM (no sentiments)',
       'Predicted high LGBM (with sentiments)',
       'Predicted high LR (no sentiments)',
       'Predicted high LR (with sentiments)'], dtype=object)

In [70]:
# use the 538 theme
alt.themes.enable('fivethirtyeight')
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

In [71]:
zoom = alt.selection_interval(encodings=["x", "y"])

palette = alt.Scale(domain=['Actual High', 'Predicted high LSTM (no sentiments)',
       'Predicted high LSTM (with sentiments)',
       'Predicted high RF (no sentiments)',
       'Predicted high RF (with sentiments)',
       'Predicted high LGBM (no sentiments)',
       'Predicted high LGBM (with sentiments)',
                           'Predicted high LR (no sentiments)','Predicted high LR (with sentiments)'],
                  range=['#330000', '#FF0000', '#0000FF', '#00FF00', '#FF8000','#663300','#808080', '#00FFFF', '#FF00FF'])

nearest = alt.selection_point(nearest=True, on='mouseover', clear='mouseout',
                        fields=['date'], empty=False)


minimap = (
    alt.Chart(melted_df)
    .mark_line()
    .add_params(zoom)
    .encode(
        x="date:T",
        y="High price:Q",
        color=alt.condition(zoom, "Method", alt.value("darkgrey")),
    )
    .properties(
        width=200,
        height=200,
        title="IPCALAB 'high'price"))

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(melted_df).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_params(
    nearest
)

columns = sorted(melted_df.Method.unique())
selection = alt.selection_single(
    fields=['date'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)

detail = (
    alt.Chart(melted_df)
    .mark_line()
    .encode(
        alt.X("date:T").scale(domain={"param": zoom.name, "encoding": "x"}),
        alt.Y("High price:Q").scale(domain={"param": zoom.name, "encoding": "y"}),
        color = alt.Color('Method:N',scale=palette, legend=alt.Legend(labelLimit = 400)),
    )
    .properties(width=600, height=400, title="IPCALAB 'high' stock price prediction results -- detail view")
)

points = detail.mark_point().transform_filter(selection)

rule = detail.transform_pivot(
    'Method', value='High price', groupby=['date']
).mark_rule().encode(
    opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip(c, type='quantitative') for c in columns]
).add_selection(selection)

(detail | minimap).configure_legend(
    orient='none',direction = 'vertical',legendX = 650,  legendY = 250, offset=-200,
    symbolDirection='vertical',
        titleFontSize=16,
        labelFontSize=14
    ).configure_title(fontSize=18)



/Users/shamil/opt/anaconda3/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/shamil/opt/anaconda3/lib/python3.9/site-packages/altair/vegalite/v5/api.py:362: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
/Users/shamil/opt/anaconda3/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)

In [72]:
source = melted_df
zoom = alt.selection_interval(encodings=["x", "y"])

minimap = (
    alt.Chart(melted_df)
    .mark_line()
    .add_params(zoom)
    .encode(
        x="date:T",
        y="High price:Q",
        color=alt.condition(zoom, "Method", alt.value("darkgrey")),
    )
    .properties(
        width=200,
        height=200,
        ))

selectors = alt.Chart(melted_df).mark_point().encode(
    x='date:T',
    opacity=alt.value(0),
).add_params(
    nearest
)

base = alt.Chart(source).encode(x='date:T')
columns = sorted(source.Method.unique())
selection = alt.selection_single(
    fields=['date'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)

lines = base.mark_line().encode(
        alt.X("date:T").scale(domain={"param": zoom.name, "encoding": "x"}),
        alt.Y("High price:Q").scale(domain={"param": zoom.name, "encoding": "y"}),
        color = alt.Color('Method:N',scale=palette, legend=alt.Legend(labelLimit = 400))
).properties(width=600, height=400, title="IPCALAB 'high' stock price prediction results -- detail view")

points = lines.mark_point().transform_filter(selection)

rule = base.transform_pivot(
    'Method', value='High price', groupby=['date']
).mark_rule().encode(
    opacity=alt.condition(selection, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip(c, type='quantitative') for c in columns]
).add_selection(selection)

(lines + points + rule | minimap).configure_legend(
    orient='none',direction = 'vertical',legendX = 650,  legendY = 250, offset=-200,
    symbolDirection='vertical',
        titleFontSize=16,
        labelFontSize=14
    ).configure_title(fontSize=18)

alt.HConcatChart(...)